# Part 1: comparing tax rates of the 2 groups

in my original project I used a very amature approach to determine if queer communities paid more or fewer taxes comapred to the general population - I did this improperly using data vizualization techniques

//add images

in this section I will attempt to use regression methods to not only determine correlation & comparison - but also to compare insights in other parameters of the groups